In [ ]:
import pandas as pd
import numpy as np
import ast

In [ ]:
movies = pd.read_csv('../input/tmdb-movie-metadata/tmdb_5000_movies.csv')
credits = pd.read_csv('../input/tmdb-movie-metadata/tmdb_5000_credits.csv')

In [ ]:
movies.head(1)

In [ ]:
movies = movies.merge(credits, on ='title')

In [ ]:
movies.shape

In [ ]:
#genres
#id
#keywords [They are tags]
#title
#overview
#cast 
#crew

movies= movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [ ]:
movies.info()

In [ ]:
movies.isnull().sum()

In [ ]:
movies.dropna(inplace=True)

In [ ]:
movies.duplicated().sum()

In [ ]:
movies.iloc[0].genres

In [ ]:
def genreConvert(obj):
    List=[]
    for i in ast.literal_eval(obj):
        List.append(i['name'])
    return List

In [ ]:
movies['genres'] = movies['genres'].apply(genreConvert)

In [ ]:
movies.head(1)

In [ ]:
def KeyConvert(obj):
    List=[]
    for i in ast.literal_eval(obj):
        List.append(i['name'])
    return List

In [ ]:
movies['keywords'] = movies['keywords'].apply(KeyConvert)

In [ ]:
def Convert(obj):
    List=[]
    counter=0
    for i in ast.literal_eval(obj):
        if counter!=3:
            List.append(i['name'])
            counter+=1
        else:
            break
    return List

In [ ]:
movies['cast'] = movies['cast'].apply(Convert)

In [ ]:
movies.head(1)

In [ ]:
def fetchJob(obj):
    List=[]
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            List.append(i['name'])
            break
    return List

In [ ]:
movies['crew'] = movies['crew'].apply(fetchJob)

In [ ]:
movies.head(1)

In [ ]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [ ]:
movies.head()

In [ ]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies.head()

In [ ]:
movies['tags'] = movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [ ]:
movies.head()

In [ ]:
new_df = movies[['movie_id','title','tags']]

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

In [ ]:
new_df['tags'][0]

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

In [ ]:
new_df.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')

In [ ]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
vectors

In [ ]:
cv.get_feature_names()

In [ ]:
import nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
    y = []
    
    for i in text.split():
        y.append(ps.stem(i))
    
    return " ".join(y)

In [ ]:
new_df['tags'] = new_df['tags'].apply(stem)

In [ ]:
stem("In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver JamesCameron")

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
similarity.shape

In [ ]:
def recommend(movie):
    movie_index = new_df[new_df['title']==movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    
    for i in movies_list:
        print(new_df.iloc[i[0]].title)
    

In [ ]:
recommend('Batman Begins')